In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE


pd.options.display.max_rows = 200
pd.options.display.max_columns = 200
sns.set_theme(style="white")

UNKNOWN = 'unknown'
RANDOM_STATE = 2021

## Import protest data

In [3]:
engine_protests = create_engine('sqlite:///../data/processed/protests.db')


with engine_protests.begin() as connection:
    protests = pd.read_sql('SELECT * FROM protests', connection)
    
protests['startdate'] = pd.to_datetime(protests.startdate)
protests['enddate'] = pd.to_datetime(protests.enddate)

## Narrow to select features

In [5]:
response_cols = ['response_accomodation', 'response_arrests', 'response_beatings', 
                 'response_crowd-dispersal', 'response_ignore', 'response_killings', 'response_shootings']

In [7]:
model_inputs = protests.drop(response_cols+['protesteridentity', 'startdate', 'enddate', 'notes', 'participants_category_original', 'country'], axis=1)
model_inputs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15208 entries, 0 to 15207
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   protest_id                          15208 non-null  int64 
 1   scode                               15208 non-null  object
 2   region                              15208 non-null  object
 3   protestnumber                       15208 non-null  int64 
 4   protesterviolence                   15208 non-null  int64 
 5   duration_days                       15208 non-null  int64 
 6   participants                        15208 non-null  int64 
 7   participants_category_manufactured  15208 non-null  object
 8   demand_labor-wage-dispute           15208 non-null  int64 
 9   demand_land-farm-issue              15208 non-null  int64 
 10  demand_police-brutality             15208 non-null  int64 
 11  demand_political-behavior/process   15208 non-null  in

## Correlation matrix

In [ ]:
# Compute the correlation matrix
corr = model_inputs.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))
plt.figure(figsize=(15, 10))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

In [ ]:
violent_cols = ['response_beatings', 'response_killings', 'response_shootings']
violent_response = protests[violent_cols].any(axis=1).astype('int')
accommodation = protests['response_accomodation']

y = violent_response #IMPORTANT LINE - CHOOSE WHICH TARGET TO USE
x_train, x_test, y_train, y_test = train_test_split(model_inputs, y, random_state=RANDOM_STATE)

In [ ]:
model_inputs.protestnumber.hist(bins=20)

In [ ]:
model_inputs.startyear.hist(bins=30)

In [ ]:
model_inputs.startmonth.hist(bins=12)

In [ ]:
model_inputs.startday.hist(bins=31)

In [ ]:
model_inputs.duration_days.hist()

In [ ]:
model_inputs.duration_days.sort_values()[:14000].hist(bins=20)

In [ ]:
model_inputs.participants.hist(bins=25)

In [ ]:
model_inputs.participants.sort_values()[:13000].hist(bins=25)

In [ ]:
model_inputs.participants_category_manufactured.value_counts()

In [ ]:
demands = ['demand_labor-wage-dispute', 'demand_land-farm-issue', 'demand_police-brutality', 'demand_political-behavior/process', 'demand_price-increases/tax-policy', 'demand_removal-of-politician', 'demand_social-restrictions']

model_inputs[demands].sum()

In [ ]:
protests[response_cols].sum()